In [1]:
import gzip
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from requests import get

Using TensorFlow backend.


In [2]:
EPOCH = 5
BATCH = 128
CLASS = 10

def _download(url_name, file_name):
    with open(file_name, "wb") as file:
        response = get(url_name)
        file.write(response.content)

def _read(m_path, l_path):
    with gzip.open(l_path, 'rb') as l_file:
        ls = np.frombuffer(l_file.read(),
                           dtype=np.uint8,
                           offset=8)

    with gzip.open(m_path,'rb') as i_file:
        length = len(ls)
        fs = np.frombuffer(i_file.read(),
                           dtype=np.uint8,
                           offset=16).reshape(length, 784).reshape(length, 28, 28, 1)

    return fs, ls

def _softmax(x):
    return np.exp(x) / (np.exp(x).sum())

def _loss(y_t, y_p, alpha):
    y_t, y_ts = y_true[: , :CLASS], y_true[: , CLASS:]
    y_p, y_ps = y_pred[: , :CLASS], y_pred[: , CLASS:]
    return alpha * logloss(y_t, y_p) + logloss(y_ts, y_ps)

def _accuracy(y_t, y_p):
    y_t = y_t[:, :CLASS]
    y_p = y_p[:, :CLASS]
    return categorical_accuracy(y_t, y_p)

def _get_model_complex():
    model = keras.Sequential()
    model.add(layers.Conv2D(filters=6,
                            kernel_size=(3, 3),
                            activation='relu',
                            input_shape=(32,32,1)))
    model.add(layers.AveragePooling2D())
    model.add(layers.Conv2D(filters=16,
                            kernel_size=(3, 3),
                            activation='relu'))
    model.add(layers.AveragePooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(units=120,
                           activation='relu'))
    model.add(layers.Dense(units=84,
                           activation='relu'))
    model.add(layers.Dense(units=10,
                           activation = 'softmax'))
    return model

def _get_model_simple():
    model = Sequential()
    model.add(layers.Flatten(input_shape=(32,32,1)))
    model.add(layers.Dense(32,
                           activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(CLASS))
    model.add(layers.Activation('softmax'))
    return model

In [3]:
_download('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', 'train-images-idx3-ubyte.gz')
_download('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', 'train-labels-idx1-ubyte.gz')
_download('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', 't10k-images-idx3-ubyte.gz')
_download('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', 't10k-labels-idx1-ubyte.gz')

In [4]:
train = {}
test = {}

train['features'], train['labels'] = _read('train-images-idx3-ubyte.gz',
                                           'train-labels-idx1-ubyte.gz')
test['features'], test['labels'] = _read('t10k-images-idx3-ubyte.gz',
                                         't10k-labels-idx1-ubyte.gz')
train['features'] = np.pad(train['features'],
                           ((0, 0), (2, 2), (2, 2), (0, 0)),
                           'constant')
test['features'] = np.pad(test['features'],
                          ((0, 0), (2, 2), (2, 2), (0, 0)),
                          'constant')

X_train, y_train = train['features'], to_categorical(train['labels'])
X_test, y_test = test['features'], to_categorical(test['labels'])

train_g = ImageDataGenerator().flow(X_train, y_train,
                                    batch_size=BATCH)
test_g = ImageDataGenerator().flow(X_test, y_test,
                                   batch_size=BATCH)

In [5]:
model = _get_model_complex()
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [6]:
model.fit_generator(train_g,
                    steps_per_epoch=X_train.shape[0] // BATCH,
                    epochs=EPOCH, 
                    validation_data=test_g,
                    validation_steps=X_test.shape[0] // BATCH, 
                    shuffle=True)

Epoch 1/5
468/468 [==============================] - 22s 46ms/step - loss: 0.3623 - accuracy: 0.9235 - val_loss: 0.1022 - val_accuracy: 0.9714
Epoch 2/5
468/468 [==============================] - 20s 43ms/step - loss: 0.0751 - accuracy: 0.9767 - val_loss: 0.1387 - val_accuracy: 0.9817
Epoch 3/5
468/468 [==============================] - 19s 41ms/step - loss: 0.0516 - accuracy: 0.9840 - val_loss: 0.0167 - val_accuracy: 0.9815
Epoch 4/5
468/468 [==============================] - 21s 45ms/step - loss: 0.0424 - accuracy: 0.9867 - val_loss: 0.0333 - val_accuracy: 0.9863
Epoch 5/5
468/468 [==============================] - 24s 51ms/step - loss: 0.0353 - accuracy: 0.9887 - val_loss: 0.0409 - val_accuracy: 0.9851


In [7]:
model_complex = keras.Model(model.input,
                            model.get_layer('dense_3').output)

In [8]:
y_train_ = np.concatenate([y_train,
                           _softmax(model_complex.predict(X_train))],
                          axis=1)
y_test_ =  np.concatenate([y_test,
                           _softmax(model_complex.predict(X_test) )],
                          axis =1)

In [9]:
model_simple = _get_model_simple()
model_simple.layers.pop()

ls = model_simple.layers[-1].output 
ps = layers.Activation('softmax')(ls)
out = layers.concatenate([ps, ps])

model_simple = keras.Model(model_simple.input, out)
model_simple.compile(loss=keras.losses.categorical_crossentropy,
                     optimizer=keras.optimizers.Adam(),
                     metrics=['accuracy'])

In [11]:
model_simple.fit(X_train, y_train_,
          batch_size=BATCH,
          epochs=EPOCH,
          verbose=1,
          validation_data=(X_test, y_test_))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 3s 44us/step - loss: 2.4699 - accuracy: 0.6844 - val_loss: 2.3933 - val_accuracy: 0.7610
Epoch 2/5
60000/60000 [==============================] - 2s 41us/step - loss: 2.4262 - accuracy: 0.7286 - val_loss: 2.3868 - val_accuracy: 0.7676
Epoch 3/5
60000/60000 [==============================] - 3s 43us/step - loss: 2.4076 - accuracy: 0.7467 - val_loss: 2.3641 - val_accuracy: 0.7900
Epoch 4/5
60000/60000 [==============================] - 3s 48us/step - loss: 2.4001 - accuracy: 0.7543 - val_loss: 2.3585 - val_accuracy: 0.7957
Epoch 5/5
60000/60000 [==============================] - 3s 49us/step - loss: 2.3973 - accuracy: 0.7571 - val_loss: 2.3595 - val_accuracy: 0.7950
